In [114]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [115]:
#Create a beautifulsoup object (soup) from the source.
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

# Find the Table in the soup...
table = soup.find('table', class_ = 'wikitable') 
#print(table)

When I print the Table, all the rows are separeted with '< tr >' , so we can exctract that information.

In [116]:
Rows = table.find_all('tr')

Pc = [] # Postal Code
Borough = []
Neigh = []
for k in Rows[1::]:
    row = k.find_all('td')
    #I use .text.split('\n')[0] to keep the text before the \n
    Pc.append(row[0].text.split('\n')[0])
    Borough.append(row[1].text.split('\n')[0])
    Neigh.append(row[2].text.split('\n')[0])


It is easier to drop the Not assigned cells with pandas after collect all the data

In [124]:
New_Neigh = []

# Replace("/", ",") and create a New list New_Neigh to store it
for N in Neigh:
    new_string = N.replace("/", ",")
    New_Neigh.append(new_string)

# Create a Dataframe and fill it with the information    
df = pd.DataFrame()
df['PostalCode'] = Pc
df['Borough'] = Borough
df['Neighborhood'] = New_Neigh 

# Drop de rows where df['Borough'] =='Not assigned'
indexBorough = df[ df['Borough'] =='Not assigned' ].index
df.drop(indexBorough , inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [130]:
# Fill where df['Neighborhood'] ==''
df.loc[df["Neighborhood"] == '','Neighborhood'] = df["Borough"]
df.head()


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [131]:
#Save the file and return the shape of the Dataframe
df.to_csv('Toronto_PC.csv')
df.shape

(103, 3)